In [13]:
#import networkx as nx
import csv
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib;
import pandas as pd 
import utm
import matplotlib.pyplot as plt 
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
from ipyleaflet import Map, FullScreenControl, Polyline, LegendControl
from datetime import datetime, timedelta, timezone
import subprocess
import numpy as np
ox.config(use_cache=True, log_console=True)
%matplotlib inline
from shapely.geometry import Point
import pyproj, math

# gt_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/reference.csv"
# gps_result_file_path = "./g2o_output/gps_lib.pos" # "/home/kcwian/Pulpit/gps_test/20210518.pos"
# g2o_result_file_path = "./g2o_output/g2o_sol.txt"
# g2o_all_result_file_path = "./g2o_output/g2o_sol_all.txt"

# gt_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/reference.csv"
# gps_result_file_path = "/home/kcwian/Pulpit/Wyniki G2O/Odaiba/4/seq_1/gps_lib_better.pos"
# g2o_result_file_path = "/home/kcwian/Pulpit/Wyniki G2O/Odaiba/4/seq_1/g2o_sol.txt"
# g2o_all_result_file_path = "/home/kcwian/Pulpit/Wyniki G2O/Odaiba/4/seq_1/g2o_sol_all.txt"

gt_file_path = "../dataset/UrbanNav/Tokyo/Shinjuku/reference.csv"
gps_result_file_path = "/home/kcwian/Pulpit/Wyniki G2O/Shinjuku/2_repeat_1_chose_as_with_the_same_params_as_Odaiba_WYniki/seq_1/gps_lib_mod.pos"
g2o_result_file_path = "/home/kcwian/Pulpit/Wyniki G2O/Shinjuku/2_repeat_1_chose_as_with_the_same_params_as_Odaiba_WYniki/seq_1/g2o_sol.txt"
g2o_all_result_file_path = "/home/kcwian/Pulpit/Wyniki G2O/Shinjuku/2_repeat_1_chose_as_with_the_same_params_as_Odaiba_WYniki/seq_1/g2o_sol_all.txt"
# gps_result_file_path = "/media/kcwian/6EF7BE2F0EBFD503/Datasets/UrbanNav/Tokyo_Data/Odaiba/rover_trimble.pos"

def filterGPSAltitude(data_in):
    data_out = []
    last_valid_altitude = data_gps_raw[0][4]
    for item in data_in:
        if abs(item[4] - last_valid_altitude) < 5:
            data_out.append(item)
            last_valid_altitude = item[4]
    return data_out

def filterG2O(data_in):
    data_out = []
    reference = data_in[1]
    prev_reference = data_in[0]
    max_dst = 50
    for item in data_in:
        if (abs(reference[2] - item[2]) < max_dst and abs(reference[3] - item[3]) < max_dst and abs(reference[2] - item[2])  < max_dst) \
        or  (abs(prev_reference[2] - item[2]) < max_dst and abs(prev_reference[3] - item[3]) < max_dst and abs(prev_reference[2] - item[2])  < max_dst):
            data_out.append(item)
            prev_reference = reference
            reference = item 
    return data_out
        

# Read ground truth data
with open(gt_file_path, 'r') as f:
    reader = csv.reader(f, delimiter=",");
    data_gt_raw = list(reader)[1:]
    for i, item in enumerate(data_gt_raw):
        data_gt_raw[i] = [float(j) for j in item]
    
# Save ground truth using UTM conversion
with open("./results/gt_utm.tum", 'w') as f:
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for item in data_gt_raw:
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[1]*3600*24*7+item[0] - leap_seconds))
        utm_res = utm.from_latlon(item[2], item[3]) # return easting, northing, zone
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, utm_res[0], utm_res[1], item[4], 0, 0, 0, 1))

# Save ground truth using ECEF conversion
# EPSG:4978-ECEF, EPSG:4326-Lat,Lon , EPSG:32650-UTM 50N (Hong Kong)           
with open("./results/gt_ecef.tum", 'w') as f:
    data_gt_lat = [item[2] for item in data_gt_raw]
    data_gt_lon = [item[3] for item in data_gt_raw]
    data_gt_alt = [item[4] for item in data_gt_raw]
    data_gt_gpd = gpd.points_from_xy(data_gt_lon, data_gt_lat, z=data_gt_alt, crs="EPSG:4326")
    data_gt_gpd_conv = data_gt_gpd.to_crs(crs="EPSG:4978")   
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37-19 # 19 - 1980, 37 - 2021
    for i, item in enumerate(data_gt_raw):
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[1]*3600*24*7+item[0] - leap_seconds))
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, data_gt_gpd_conv[i].x, data_gt_gpd_conv[i].y, data_gt_gpd_conv[i].z, 0, 0, 0, 1))

# Read GPS estimated pose
with open(gps_result_file_path, 'r') as f:
    reader = csv.reader(filter(lambda row: row[0]!='%', f), delimiter=",",quoting=csv.QUOTE_NONNUMERIC)
    data_gps_raw = list(reader)
    # data_gps_raw = filterGPSAltitude(data_gps_raw)
    
# Convert GPS estimated pose and save to UTM file
with open("./results/gps_utm.tum", 'w') as f:
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for item in data_gps_raw:
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        utm_res = utm.from_latlon(item[2], item[3]) # return easting, northing, zone
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, utm_res[0], utm_res[1], item[4], 0, 0, 0, 1))

# Convert GPS estimated pose and save to file
with open("./results/gps_ecef.tum", 'w') as f:
    data_gps_lat = [item[2] for item in data_gps_raw]
    data_gps_lon = [item[3] for item in data_gps_raw]
    data_gps_alt = [item[4] for item in data_gps_raw]
    data_gps_gpd = gpd.points_from_xy(data_gps_lon, data_gps_lat, z=data_gps_alt, crs="EPSG:4326")
    data_gps_gpd_conv = data_gps_gpd.to_crs(crs="EPSG:4978")  
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    #print(data_gps_lat[0], data_gps_lon[0])
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for i, item in enumerate(data_gps_raw):
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, data_gps_gpd_conv[i].x, data_gps_gpd_conv[i].y, data_gps_gpd_conv[i].z, 0, 0, 0, 1))

# Read g2o estimated pose
with open(g2o_result_file_path, 'r') as f:
    reader = csv.reader(f, delimiter=",",quoting=csv.QUOTE_NONNUMERIC);
    data_g2o_raw = list(reader)
    #data_g2o_raw = filterG2O(data_g2o_raw)
    
# Convert g2o estimated pose and save to file
with open("./results/g2o_ecef.tum", 'w') as f:
    data_g2o_x = [ item[2] for item in data_g2o_raw]
    data_g2o_y = [ item[3] for item in data_g2o_raw]
    data_g2o_z = [ item[4] for item in data_g2o_raw]
    data_g2o_qx = [ item[5] for item in data_g2o_raw]
    data_g2o_qy = [ item[6] for item in data_g2o_raw]
    data_g2o_qz = [ item[7] for item in data_g2o_raw]
    data_g2o_qw = [ item[8] for item in data_g2o_raw]
    data_g2o_gpd = gpd.points_from_xy(data_g2o_x, data_g2o_y, z=data_g2o_z, crs="EPSG:4978")
    data_g2o_gpd_conv = data_g2o_gpd.to_crs(crs="EPSG:4326")
    #print(data_g2o_gpd_conv[0].y, data_g2o_gpd_conv[0].x) 
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for i, item in enumerate(data_g2o_raw):
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, data_g2o_x[i], data_g2o_y[i], data_g2o_z[i], data_g2o_qx[i], data_g2o_qy[i], data_g2o_qz[i], data_g2o_qw[i]))

        # Read g2o estimated pose
with open(g2o_all_result_file_path, 'r') as f:
    reader = csv.reader(f, delimiter=",",quoting=csv.QUOTE_NONNUMERIC);
    data_g2o_all_raw = list(reader)
    #data_g2o_raw = filterG2O(data_g2o_raw)
    
# Convert g2o estimated pose and save to file
with open("./results/g2o_all_ecef.tum", 'w') as f:
    data_g2o_all_x = [ item[2] for item in data_g2o_all_raw]
    data_g2o_all_y = [ item[3] for item in data_g2o_all_raw]
    data_g2o_all_z = [ item[4] for item in data_g2o_all_raw]
    data_g2o_all_qx = [ item[5] for item in data_g2o_all_raw]
    data_g2o_all_qy = [ item[6] for item in data_g2o_all_raw]
    data_g2o_all_qz = [ item[7] for item in data_g2o_all_raw]
    data_g2o_all_qw = [ item[8] for item in data_g2o_all_raw]
    data_g2o_all_gpd = gpd.points_from_xy(data_g2o_all_x, data_g2o_all_y, z=data_g2o_all_z, crs="EPSG:4978")
    data_g2o_all_gpd_conv = data_g2o_all_gpd.to_crs(crs="EPSG:4326")
    #print(data_g2o_all_gpd_conv[0].y, data_g2o_all_gpd_conv[0].x) 
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for i, item in enumerate(data_g2o_all_raw):
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, data_g2o_all_x[i], data_g2o_all_y[i], data_g2o_all_z[i], data_g2o_all_qx[i], data_g2o_all_qy[i], data_g2o_all_qz[i], data_g2o_all_qw[i]))

# Call evo_ape evaluation
f = open("./results/evo_ape_gps.txt", "w")
subprocess.call("rm ape_gps.pdf", shell=True, cwd="./results")
subprocess.call("evo_ape tum gt_ecef.tum gps_ecef.tum -va --save_plot ape_gps.pdf", stdout=f, shell=True, cwd="./results")
f.close()

# Call evo_ape evaluation
f = open("./results/evo_ape_g2o.txt", "w")
subprocess.call("rm ape_g2o.pdf", shell=True, cwd="./results")
subprocess.call("evo_ape tum gt_ecef.tum g2o_ecef.tum -va --save_plot ape_g2o.pdf", stdout=f, shell=True, cwd="./results")
f.close()

# Call evo_ape evaluation
f = open("./results/evo_ape_g2o_all.txt", "w")
subprocess.call("rm ape_g2o_all.pdf", shell=True, cwd="./results")
subprocess.call("evo_ape tum gt_ecef.tum g2o_all_ecef.tum -va --save_plot ape_g2o_all.pdf", stdout=f, shell=True, cwd="./results")
f.close()

print("GPS")
with open("./results/evo_ape_gps.txt", 'r') as f:
    lines = f.readlines()
    print(lines[18].strip())
    for line in lines[24:31]:
        print(line.strip())
        
print("\ng2o saved online")
with open("./results/evo_ape_g2o.txt", 'r') as f:
    lines = f.readlines()
    print(lines[18].strip())
    for line in lines[24:31]:
        print(line.strip())
        
print("\ng2o saved at the end")
with open("./results/evo_ape_g2o_all.txt", 'r') as f:
    lines = f.readlines()
    print(lines[18].strip())
    for line in lines[24:31]:
        print(line.strip())
    
# Generate XYZ Plot
subprocess.call("rm xyz.pdf", shell=True, cwd="./results")
subprocess.call("evo_traj tum gps_utm.tum --ref=gt_utm.tum --save_plot xyz.pdf", shell=True, cwd="./results")

# Add map visualization
# Ground truth
gt_line = Polyline(
    locations=[[x,y] for x,y in zip(data_gt_lat[:],data_gt_lon[:])],
    color="black",
    fill=False,
    opacity = 1,
    weight=6,
)
gps_line= Polyline(
    locations=[[x,y] for x,y in zip(data_gps_lat[:],data_gps_lon[:])],
    color="green",
    fill=False,
    opacity=1,
    weight=4,
)
g2o_line = Polyline(
    locations=[[item.y, item.x] for item in data_g2o_gpd_conv[:]],
    color="yellow",
    fill=False,
    opacity=1,
    weight=4,
)
g2o_all_line = Polyline(
    locations=[[item.y, item.x] for item in data_g2o_all_gpd_conv[:]],
    color="red",
    fill=False,
    opacity=1,
    weight = 3,
)
m = Map(center=[data_gps_lat[0], data_gps_lon[0]], basemap=basemaps.OpenStreetMap.Mapnik, zoom=15)
m.add_control(FullScreenControl())
m.add_layer(gt_line)
m.add_layer(gps_line)
# m.add_layer(g2o_line)
m.add_layer(g2o_all_line)
m.save("./results/map.html")
print("gps: " , len(data_gps_lat))
print("g2o: " , len(data_g2o_gpd_conv))
legend = LegendControl({"reference" : "black", "RTKLIB":"green", "g2o (LOAM + GNSS)":"red"}, name="Legend", position="topright")
m.add_control(legend)
m

GPS


IndexError: list index out of range